<a href="https://colab.research.google.com/github/nirmalghimire/DDP_Achievement-Gap/blob/main/Covid_impact_2022_OECD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages
!pip install pyreadr shap matplotlib seaborn scikit-learn pandas numpy missingno

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import pyreadr
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from tabulate import tabulate
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.7/411.7 kB 11.1 MB/s eta 0:00:00


In [ ]:
# Upload the data from your local machine to Google Colab
from google.colab import files
uploaded = files.upload()

Saving merged_school_data_2022.RData to merged_school_data_2022.RData


In [ ]:
# Load the RData file
result = pyreadr.read_r('merged_school_data_2022.RData')
df = result[list(result.keys())[0]]  # Get the first element in the dictionary

In [ ]:
# Display basic information about the dataset
print(f"Dataset Shape: {df.shape}")
print("Column names:", df.columns.tolist())
print(df.head())

Dataset Shape: (21629, 18)
Column names: ['CNT', 'CNTRYID', 'CNTSCHID', 'OECD', 'PRIVATESCH', 'SCHLTYPE', 'PROBSCRI', 'SCPREPAP', 'SCPREPBP', 'DIGPREP', 'DaysClosed_COVID', 'Remote_Teaching', 'Self_Study', 'Class_Cancelled', 'W_FSTUWT_SCH_N', 'sch_reading_score', 'sch_math_score', 'sch_science_score']
   CNT  CNTRYID  CNTSCHID  OECD PRIVATESCH SCHLTYPE  PROBSCRI  SCPREPAP  \
0  ALB      8.0  800001.0   0.0     Public        3    0.7965    0.8462   
1  ALB      8.0  800002.0   0.0     Public        3   -0.5687    0.8462   
2  ALB      8.0  800003.0   0.0    Private        1    0.1896   -0.8711   
3  ALB      8.0  800004.0   0.0     Public        3       NaN       NaN   
4  ALB      8.0  800005.0   0.0     Public        3    0.6649    0.8462   

   SCPREPBP  DIGPREP  DaysClosed_COVID  Remote_Teaching  Self_Study  \
0   -0.8314   0.5908              90.0              5.0         5.0   
1   -0.8314  -0.3475              70.0              5.0         1.0   
2   -0.8314   0.4409             

In [ ]:
# Define features and target variable
features = [
    'SCHLTYPE', 'OECD', 'DIGPREP', 'PROBSCRI', 'SCPREPAP', 'SCPREPBP',
    'DaysClosed_COVID', 'Remote_Teaching', 'Self_Study', 'Class_Cancelled'
]
target = 'sch_reading_score'

# Create a new dataframe with only the features and target
data = df[features + [target]].copy()

In [ ]:
# Check missing data
missing_data = data.isnull().sum()
print("\nMissing data per column:")
print(missing_data)
missing_percent = (missing_data / len(data)) * 100
print("\nMissing data percentage:")
print(missing_percent)

# Check the distribution of categorical variables
print("\nSCHLTYPE value counts:")
print(data['SCHLTYPE'].value_counts(dropna=False))
print("\nOECD value counts:")
print(data['OECD'].value_counts(dropna=False))


Missing data per column:
SCHLTYPE              1622
OECD                     0
DIGPREP               4050
PROBSCRI              6380
SCPREPAP             10725
SCPREPBP              5776
DaysClosed_COVID      3139
Remote_Teaching       6142
Self_Study            6423
Class_Cancelled       6539
sch_reading_score        0
dtype: int64

Missing data percentage:
SCHLTYPE              7.499191
OECD                  0.000000
DIGPREP              18.724860
PROBSCRI             29.497434
SCPREPAP             49.586204
SCPREPBP             26.704887
DaysClosed_COVID     14.512922
Remote_Teaching      28.397060
Self_Study           29.696241
Class_Cancelled      30.232558
sch_reading_score     0.000000
dtype: float64

SCHLTYPE value counts:
SCHLTYPE
3      16017
1       2277
2       1713
NaN     1622
Name: count, dtype: int64

OECD value counts:
OECD
1.0    11108
0.0    10521
Name: count, dtype: int64


# =====================
# PART 1: OECD vs. NON-OECD ANALYSIS
# =====================

In [ ]:
print("\n\n" + "="*50)
print("PART 1: OECD vs. NON-OECD ANALYSIS")
print("="*50)

def analyze_oecd_subset(data, oecd_status, subset_name):
    print(f"\n\n===== ANALYZING {subset_name} =====")

    # Filter data for the specific OECD status
    subset_data = data[data['OECD'] == oecd_status].copy()
    print(f"Number of schools in {subset_name}: {len(subset_data)}")

    # Handle missing data in target variable
    subset_data = subset_data.dropna(subset=[target])
    print(f"Schools with valid target in {subset_name}: {len(subset_data)}")

    if len(subset_data) < 100:
        print(f"Not enough data for {subset_name} analysis.")
        return None

    # Remove OECD from features since we're analyzing by OECD status
    subset_features = [f for f in features if f != 'OECD']

    # Define categorical and numerical features
    categorical_features = ['SCHLTYPE']
    numerical_features = [f for f in subset_features if f != 'SCHLTYPE']

    # Prepare X and y
    X = subset_data[subset_features]
    y = subset_data[target]

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create preprocessing pipeline with imputation
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder(drop='first', sparse_output=False))
            ]), categorical_features),
            ('num', IterativeImputer(random_state=42), numerical_features)
        ])

    # Create a pipeline with preprocessing and random forest
    rf_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(n_estimators=500, max_depth=15, random_state=42, n_jobs=-1))
    ])

    # Fit the model
    print(f"Training Random Forest model for {subset_name}...")
    try:
        rf_pipeline.fit(X_train, y_train)
        print(f"Model training completed for {subset_name}")
    except Exception as e:
        print(f"Error during model training: {str(e)}")
        return None

    # Get feature names after one-hot encoding
    try:
        feature_names = []
        # Get one-hot encoded feature names for categorical features
        ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']
        feature_names.extend(ohe.get_feature_names_out(categorical_features))
        # Add numerical feature names
        feature_names.extend(numerical_features)
        print(f"Feature names extracted: {len(feature_names)} features")
    except Exception as e:
        print(f"Error getting feature names: {str(e)}")
        return None

    # Make predictions and evaluate the model
    try:
        y_pred = rf_pipeline.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mse)

        print(f"{subset_name} - Mean Squared Error: {mse:.2f}")
        print(f"{subset_name} - R² Score: {r2:.4f}")
        print(f"{subset_name} - RMSE: {rmse:.2f}")
    except Exception as e:
        print(f"Error during model evaluation: {str(e)}")
        return None

    # Create model performance table
    model_performance = {
        'Model': f"Random Forest ({subset_name})",
        'Mean Squared Error': f'{mse:.2f}',
        'R² Score': f'{r2:.4f}',
        'RMSE': f'{rmse:.2f}',
        'Sample Size': len(X_test)
    }

    # Print the table
    print(f"\n{subset_name} Model Performance:")
    print(tabulate([model_performance], headers='keys', tablefmt='simple'))

    # Save performance to file
    with open(f'{subset_name.replace(" ", "_")}_performance.csv', 'w') as f:
        f.write(tabulate([model_performance], headers='keys', tablefmt='csv'))

    # Get the RandomForest model from the pipeline
    rf_model = rf_pipeline.named_steps['regressor']

    # Get feature importances
    try:
        importances = rf_model.feature_importances_
        print(f"Feature importances extracted: {len(importances)} values")

        # Sort feature importances
        indices = np.argsort(importances)[::-1]

        # Create a feature importance table
        fi_table = []
        cumulative = 0

        for i in indices:
            clean_name = feature_names[i].replace('_', ' ').capitalize()
            if clean_name.startswith('SCHLTYPE'):
                clean_name = clean_name.replace('SCHLTYPE_', 'School Type: ')

            cumulative += importances[i]
            fi_table.append({
                'Feature': clean_name,
                'Importance': f'{importances[i]:.4f}',
                'Cumulative %': f'{cumulative*100:.2f}%'
            })

        # Print the feature importance table
        print(f"\n{subset_name} Feature Importance:")
        print(tabulate(fi_table, headers='keys', tablefmt='simple'))

        # Save feature importance to file
        with open(f'{subset_name.replace(" ", "_")}_feature_importance.csv', 'w') as f:
            f.write(tabulate(fi_table, headers='keys', tablefmt='csv'))
    except Exception as e:
        print(f"Error calculating feature importance: {str(e)}")
        fi_table = []

    # SHAP analysis
    shap_table = []
    try:
        print(f"\nCalculating SHAP values for {subset_name}...")

        # Transform the test data
        X_test_transformed = preprocessor.transform(X_test)

        # Use a sample if the test set is large
        max_shap_samples = 2000
        if len(X_test) > max_shap_samples:
            print(f"Sampling {max_shap_samples} out of {len(X_test)} for SHAP analysis")
            sample_indices = np.random.choice(len(X_test), max_shap_samples, replace=False)
            X_test_sample = X_test_transformed[sample_indices]
        else:
            X_test_sample = X_test_transformed

        # Create an explainer
        explainer = shap.TreeExplainer(rf_model)

        # Calculate SHAP values
        shap_values = explainer.shap_values(X_test_sample)
        print(f"SHAP values calculated with shape: {shap_values.shape}")

        # Create a SHAP values table
        mean_abs_shap = np.abs(shap_values).mean(0)
        top_indices = np.argsort(mean_abs_shap)[::-1]

        for i in top_indices:
            clean_name = feature_names[i].replace('_', ' ').capitalize()
            if clean_name.startswith('SCHLTYPE'):
                clean_name = clean_name.replace('SCHLTYPE_', 'School Type: ')

            shap_table.append({
                'Feature': clean_name,
                'Mean |SHAP Value|': f'{mean_abs_shap[i]:.4f}',
                'Min SHAP': f'{np.min(shap_values, axis=0)[i]:.4f}',
                'Max SHAP': f'{np.max(shap_values, axis=0)[i]:.4f}'
            })

        # Print the SHAP values table
        print(f"\n{subset_name} SHAP Values:")
        print(tabulate(shap_table[:10], headers='keys', tablefmt='simple'))

        # Save SHAP values to file
        with open(f'{subset_name.replace(" ", "_")}_shap_values.csv', 'w') as f:
            f.write(tabulate(shap_table[:10], headers='keys', tablefmt='csv'))
    except Exception as e:
        print(f"Error during SHAP analysis: {str(e)}")

    # Return the results
    return {
        'subset_name': subset_name,
        'model_performance': model_performance,
        'feature_importance': fi_table,
        'shap_values': shap_table[:10]
    }

# Run OECD status analyses
print("\nRunning analysis for OECD countries...")
oecd_results = analyze_oecd_subset(data, 1, "OECD Countries")

print("\nRunning analysis for non-OECD countries...")
non_oecd_results = analyze_oecd_subset(data, 0, "Non-OECD Countries")

# Compare OECD and non-OECD results
if oecd_results and non_oecd_results:
    # Compare model performance
    oecd_performance = [
        {'Subset': 'OECD Countries',
         'MSE': oecd_results['model_performance']['Mean Squared Error'],
         'R²': oecd_results['model_performance']['R² Score'],
         'RMSE': oecd_results['model_performance']['RMSE'],
         'Sample Size': oecd_results['model_performance']['Sample Size']},
        {'Subset': 'Non-OECD Countries',
         'MSE': non_oecd_results['model_performance']['Mean Squared Error'],
         'R²': non_oecd_results['model_performance']['R² Score'],
         'RMSE': non_oecd_results['model_performance']['RMSE'],
         'Sample Size': non_oecd_results['model_performance']['Sample Size']}
    ]

    print("\n\nComparison of OECD vs. Non-OECD Performance:")
    print(tabulate(oecd_performance, headers='keys', tablefmt='simple'))

    # Save comparison to file
    with open('oecd_performance_comparison.csv', 'w') as f:
        f.write(tabulate(oecd_performance, headers='keys', tablefmt='csv'))

    # Compare top 5 features
    top_features_comparison = []

    # Get top 5 features for OECD
    oecd_top = [f['Feature'] for f in oecd_results['feature_importance'][:5]]
    non_oecd_top = [f['Feature'] for f in non_oecd_results['feature_importance'][:5]]

    top_features_comparison.append({
        'Subset': 'OECD Countries',
        'Top Feature 1': oecd_top[0],
        'Top Feature 2': oecd_top[1],
        'Top Feature 3': oecd_top[2],
        'Top Feature 4': oecd_top[3],
        'Top Feature 5': oecd_top[4]
    })

    top_features_comparison.append({
        'Subset': 'Non-OECD Countries',
        'Top Feature 1': non_oecd_top[0],
        'Top Feature 2': non_oecd_top[1],
        'Top Feature 3': non_oecd_top[2],
        'Top Feature 4': non_oecd_top[3],
        'Top Feature 5': non_oecd_top[4]
    })

    print("\n\nTop Features Comparison (OECD vs. Non-OECD):")
    print(tabulate(top_features_comparison, headers='keys', tablefmt='simple'))

    # Save comparison to file
    with open('oecd_top_features_comparison.csv', 'w') as f:
        f.write(tabulate(top_features_comparison, headers='keys', tablefmt='csv'))

    # Compare SHAP values
    common_features = set([item['Feature'] for item in oecd_results['shap_values'][:10]]).intersection(
                      set([item['Feature'] for item in non_oecd_results['shap_values'][:10]]))

    shap_comparison = []

    for feature in common_features:
        oecd_shap = next((item['Mean |SHAP Value|'] for item in oecd_results['shap_values'] if item['Feature'] == feature), 'N/A')
        non_oecd_shap = next((item['Mean |SHAP Value|'] for item in non_oecd_results['shap_values'] if item['Feature'] == feature), 'N/A')

        shap_comparison.append({
            'Feature': feature,
            'OECD Mean |SHAP|': oecd_shap,
            'Non-OECD Mean |SHAP|': non_oecd_shap
        })

    print("\n\nSHAP Value Comparison for Common Top Features:")
    print(tabulate(shap_comparison, headers='keys', tablefmt='simple'))

    # Save comparison to file
    with open('oecd_shap_comparison.csv', 'w') as f:
        f.write(tabulate(shap_comparison, headers='keys', tablefmt='csv'))



PART 1: OECD vs. NON-OECD ANALYSIS

Running analysis for OECD countries...


===== ANALYZING OECD Countries =====
Number of schools in OECD Countries: 11108
Schools with valid target in OECD Countries: 11108
Training Random Forest model for OECD Countries...


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Model training completed for OECD Countries
Feature names extracted: 10 features


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


OECD Countries - Mean Squared Error: 3633.59
OECD Countries - R² Score: 0.1249
OECD Countries - RMSE: 60.28

OECD Countries Model Performance:
Model                             Mean Squared Error    R² Score    RMSE    Sample Size
------------------------------  --------------------  ----------  ------  -------------
Random Forest (OECD Countries)               3633.59      0.1249   60.28           2222
Feature importances extracted: 10 values

OECD Countries Feature Importance:
Feature             Importance  Cumulative %
----------------  ------------  --------------
Probscri                0.2206  22.06%
Daysclosed covid        0.1761  39.67%
Digprep                 0.1709  56.76%
Scprepap                0.135   70.26%
Scprepbp                0.094   79.66%
Self study              0.064   86.05%
Remote teaching         0.0538  91.43%
Class cancelled         0.0493  96.36%
Schltype 3              0.0293  99.29%
Schltype 2              0.0071  100.00%

Calculating SHAP values for OECD

...

Model training completed for Non-OECD Countries
Feature names extracted: 10 features


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Non-OECD Countries - Mean Squared Error: 4120.96
Non-OECD Countries - R² Score: 0.2038
Non-OECD Countries - RMSE: 64.19

Non-OECD Countries Model Performance:
Model                                 Mean Squared Error    R² Score    RMSE    Sample Size
----------------------------------  --------------------  ----------  ------  -------------
Random Forest (Non-OECD Countries)               4120.96      0.2038   64.19           2105
Feature importances extracted: 10 values

Non-OECD Countries Feature Importance:
Feature             Importance  Cumulative %
----------------  ------------  --------------
Probscri                0.2452  24.52%
Digprep                 0.1448  38.99%
Daysclosed covid        0.1387  52.87%
Scprepap                0.1346  66.33%
Schltype 3              0.0885  75.19%
Scprepbp                0.0766  82.84%
Class cancelled         0.0641  89.25%
Self study              0.0583  95.08%
Remote teaching         0.0418  99.26%
Schltype 2              0.0074  100.00%



/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


SHAP values calculated with shape: (2000, 10)

Non-OECD Countries SHAP Values:
Feature             Mean |SHAP Value|    Min SHAP    Max SHAP
----------------  -------------------  ----------  ----------
Probscri                      14.7146    -47.3883     54.9427
Schltype 3                    11.9114    -17.2083     56.7086
Scprepap                       7.2645    -29.0301     24.6353
Class cancelled                6.5094    -39.2935     19.5523
Daysclosed covid               4.1982    -28.9257     29.8733
Remote teaching                3.4903    -38.986       9.4508
Self study                     3.2961    -18.1511     12.4242
Digprep                        2.5202    -10.9673     28.4983
Scprepbp                       1.9766    -10.094      15.0699
Schltype 2                     0.2053     -5.8166     11.4557


Comparison of OECD vs. Non-OECD Performance:
Subset                  MSE      R²    RMSE    Sample Size
------------------  -------  ------  ------  -------------
OECD Countri

In [ ]:
# Function to analyze each school type subset (keeping OECD as a predictor)
def analyze_school_type(data, schltype_value, subset_name):
    print(f"\n\n===== ANALYZING {subset_name} =====")

    # Filter data for the specific school type
    subset_data = data[data['SCHLTYPE'] == schltype_value].copy()
    print(f"Number of schools in {subset_name}: {len(subset_data)}")

    # Handle missing data in target variable
    subset_data = subset_data.dropna(subset=[target])
    print(f"Schools with valid target in {subset_name}: {len(subset_data)}")

    if len(subset_data) < 100:
        print(f"Not enough data for {subset_name} analysis.")
        return None

    # Remove SCHLTYPE from features since we're analyzing by school type
    # But keep OECD as a predictor
    subset_features = [f for f in features if f != 'SCHLTYPE']

    # Prepare X and y
    X = subset_data[subset_features]
    y = subset_data[target]

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create preprocessing pipeline with imputation
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', IterativeImputer(random_state=42), subset_features)
        ])

    # Create a pipeline with preprocessing and random forest
    rf_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(n_estimators=500, max_depth=15, random_state=42, n_jobs=-1))
    ])

    # Fit the model
    print(f"Training Random Forest model for {subset_name}...")
    rf_pipeline.fit(X_train, y_train)

    # Get feature names
    feature_names = subset_features

    # Make predictions and evaluate the model
    y_pred = rf_pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mse)

    print(f"{subset_name} - Mean Squared Error: {mse:.2f}")
    print(f"{subset_name} - R² Score: {r2:.4f}")
    print(f"{subset_name} - RMSE: {rmse:.2f}")

    # Create model performance table
    model_performance = {
        'Model': f"Random Forest ({subset_name})",
        'Mean Squared Error': f'{mse:.2f}',
        'R² Score': f'{r2:.4f}',
        'RMSE': f'{rmse:.2f}',
        'Sample Size': len(X_test)
    }

    # Print the table
    print(f"\n{subset_name} Model Performance:")
    print(tabulate([model_performance], headers='keys', tablefmt='simple'))

    # Get the RandomForest model from the pipeline
    rf_model = rf_pipeline.named_steps['regressor']

    # Get feature importances
    importances = rf_model.feature_importances_

    # Sort feature importances
    indices = np.argsort(importances)[::-1]

    # Create a feature importance table
    fi_table = []
    cumulative = 0

    for i in indices:
        clean_name = feature_names[i].replace('_', ' ').capitalize()
        cumulative += importances[i]
        fi_table.append({
            'Feature': clean_name,
            'Importance': f'{importances[i]:.4f}',
            'Cumulative %': f'{cumulative*100:.2f}%'
        })

    # Print the feature importance table
    print(f"\n{subset_name} Feature Importance:")
    print(tabulate(fi_table, headers='keys', tablefmt='simple'))

    # SHAP analysis
    print(f"\nCalculating SHAP values for {subset_name}...")

    # Transform the test data
    X_test_transformed = preprocessor.transform(X_test)

    # Create an explainer
    explainer = shap.TreeExplainer(rf_model)

    # Calculate SHAP values
    shap_values = explainer.shap_values(X_test_transformed)

    # Create a SHAP values table
    mean_abs_shap = np.abs(shap_values).mean(0)
    top_indices = np.argsort(mean_abs_shap)[::-1]

    shap_table = []
    for i in top_indices:
        clean_name = feature_names[i].replace('_', ' ').capitalize()
        shap_table.append({
            'Feature': clean_name,
            'Mean |SHAP Value|': f'{mean_abs_shap[i]:.4f}',
            'Min SHAP': f'{np.min(shap_values, axis=0)[i]:.4f}',
            'Max SHAP': f'{np.max(shap_values, axis=0)[i]:.4f}'
        })

    # Print the SHAP values table
    print(f"\n{subset_name} SHAP Values:")
    print(tabulate(shap_table[:10], headers='keys', tablefmt='simple'))

    # Return the results
    return {
        'subset_name': subset_name,
        'model_performance': model_performance,
        'feature_importance': fi_table,
        'shap_values': shap_table[:10],
        'feature_names': feature_names,
        'shap_vals': shap_values,
        'X_test_transformed': X_test_transformed,
        'explainer': explainer,
        'rf_model': rf_model
    }

# Run analysis for each school type
# Make sure to use integer values that match the data
private_independent_results = analyze_school_type(data, 1, "Private Independent Schools")
private_govt_dependent_results = analyze_school_type(data, 2, "Private Government-Dependent Schools")
public_schools_results = analyze_school_type(data, 3, "Public Schools")

# Collect all model performance results for comparison
all_model_results = []
for result in [
    {'name': 'Full Dataset', 'mse': 3812.79, 'r2': 0.3772, 'rmse': 61.75, 'sample_size': 4326},
    private_independent_results,
    private_govt_dependent_results,
    public_schools_results
]:
    if result:
        if isinstance(result, dict) and 'model_performance' in result:
            # Extract from results dictionary
            perf = result['model_performance']
            all_model_results.append({
                'Subset': result['subset_name'],
                'MSE': perf['Mean Squared Error'],
                'R²': perf['R² Score'],
                'RMSE': perf['RMSE'],
                'Sample Size': perf['Sample Size']
            })
        else:
            # Directly use provided dictionary
            all_model_results.append({
                'Subset': result['name'],
                'MSE': f"{result['mse']:.2f}",
                'R²': f"{result['r2']:.4f}",
                'RMSE': f"{result['rmse']:.2f}",
                'Sample Size': result['sample_size']
            })

# Print the comparative table
print("\n\nComparative Model Performance Across School Types:")
print(tabulate(all_model_results, headers='keys', tablefmt='simple'))

# If all analyses were successful, compare feature importance across school types
if private_independent_results and private_govt_dependent_results and public_schools_results:
    # Get top 5 features for each school type
    feature_comparison = []

    for result in [private_independent_results, private_govt_dependent_results, public_schools_results]:
        top_features = []
        for i, feature in enumerate(result['feature_importance']):
            if i < 5:  # Top 5 features
                top_features.append(feature['Feature'])
        feature_comparison.append({
            'School Type': result['subset_name'],
            'Top Feature 1': top_features[0],
            'Top Feature 2': top_features[1],
            'Top Feature 3': top_features[2],
            'Top Feature 4': top_features[3],
            'Top Feature 5': top_features[4]
        })

    print("\n\nTop Features by School Type:")
    print(tabulate(feature_comparison, headers='keys', tablefmt='simple'))

    # Create a comparison of OECD importance across school types
    oecd_comparison = []

    for result in [private_independent_results, private_govt_dependent_results, public_schools_results]:
        # Find OECD in feature importance table
        oecd_importance = '0.0000'
        oecd_rank = 'Not Found'

        for i, feature in enumerate(result['feature_importance']):
            if feature['Feature'].lower() == 'oecd':
                oecd_importance = feature['Importance']
                oecd_rank = str(i + 1)
                break

        # Find OECD in SHAP values table
        oecd_shap = '0.0000'
        for feature in result['shap_values']:
            if feature['Feature'].lower() == 'oecd':
                oecd_shap = feature['Mean |SHAP Value|']
                break

        oecd_comparison.append({
            'School Type': result['subset_name'],
            'OECD Importance': oecd_importance,
            'OECD Rank': oecd_rank,
            'OECD Mean |SHAP|': oecd_shap
        })

    print("\n\nOECD Importance by School Type:")
    print(tabulate(oecd_comparison, headers='keys', tablefmt='simple'))



===== ANALYZING Private Independent Schools =====
Number of schools in Private Independent Schools: 0
Schools with valid target in Private Independent Schools: 0
Not enough data for Private Independent Schools analysis.


===== ANALYZING Private Government-Dependent Schools =====
Number of schools in Private Government-Dependent Schools: 0
Schools with valid target in Private Government-Dependent Schools: 0
Not enough data for Private Government-Dependent Schools analysis.


===== ANALYZING Public Schools =====
Number of schools in Public Schools: 0
Schools with valid target in Public Schools: 0
Not enough data for Public Schools analysis.


Comparative Model Performance Across School Types:
Subset            MSE      R²    RMSE    Sample Size
------------  -------  ------  ------  -------------
Full Dataset  3812.79  0.3772   61.75           4326
